In [96]:
import pandas as pd
import plotly.express as px

# Graphs:

Hay que usar el playerId para unir los datos de los jugadores con lo de wyscout


### Player info
- nombre (dropdown que se puede buscar)
- equipo (dropdown)
- edad
- peso
- posición (la de wyscout o de *clusters idealmente)
- *costo

### Player stats

(se necesita un .csv con todas estas estadísticas aparte de )

- xT
- VAEP
- agregados de todo
- *minutos jugados

In [97]:
df_eng = pd.read_csv('./../../data/wyscout_tabular/england.csv')
df_ger = pd.read_csv('./../../data/wyscout_tabular/germany.csv')
df_ita = pd.read_csv('./../../data/wyscout_tabular/italy.csv')
df_fra = pd.read_csv('./../../data/wyscout_tabular/france.csv')
df_spa = pd.read_csv('./../../data/wyscout_tabular/spain.csv')
#df_wc = pd.read_csv('./../../data/wyscout_tabular/events_World_Cup.csv')

dfs = [df_eng, df_ger, df_ita, df_fra, df_spa]

df_events = pd.concat(dfs)

In [98]:
df_p_eng = pd.read_csv('./../../data/datos_scrappeados/players_generalEngland-Football.csv')
df_p_ger = pd.read_csv('./../../data/datos_scrappeados/players_generalGermany-Football.csv')
df_p_ita = pd.read_csv('./../../data/datos_scrappeados/players_generalItaly-Football.csv')
df_p_fra = pd.read_csv('./../../data/datos_scrappeados/players_generalFrance-Football.csv')
df_p_spa = pd.read_csv('./../../data/datos_scrappeados/players_generalSpain-Football.csv')

In [99]:
player_stats = [df_p_eng, df_p_ger, df_p_ita, df_p_fra, df_p_spa]
player_stats = pd.concat(player_stats)

In [100]:
df_players = pd.read_json('./../../data/wyscout/players.json', encoding='utf8')
df_teams = pd.read_json('./../../data/wyscout/Teams/teams.json', encoding='utf8')
df_comps = pd.read_json('./../../data/wyscout/Competitions/competitions.json', encoding='utf8')

df_players[['name', 'id', 'alpha3code', 'alpha2code']] = pd.json_normalize(df_players['passportArea'])
df_players['role'] = pd.json_normalize(df_players['role'])['name']
df_players[['country', 'cou_code']] = pd.json_normalize(df_players['birthArea'])[['name', 'alpha2code']]

df_players = df_players.drop(columns=['passportArea', 'birthArea'])

In [101]:
#fix encoding
df_teams['officialName'] = df_teams['officialName'].apply(lambda x: x.encode().decode('unicode_escape'))
df_teams['name'] = df_teams['name'].apply(lambda x: x.encode().decode('unicode_escape'))

df_players['firstName'] = df_players['firstName'].apply(lambda x: x.encode().decode('unicode_escape'))
df_players['lastName'] = df_players['lastName'].apply(lambda x: x.encode().decode('unicode_escape'))
df_players['middleName'] = df_players['middleName'].apply(lambda x: x.encode().decode('unicode_escape'))

In [102]:
df_players = df_players[['firstName', 'middleName', 'lastName', 'currentTeamId', 'weight', 
                         'height', 'foot', 'country', 'cou_code', 'role']]

#some players dont have club assigned. It's because of players from the wold cup that don't have a team on the top 5 leagues
df_all_info = df_players.merge(df_teams[['city', 'name', 'officialName', 'wyId']], left_on='currentTeamId', right_on='wyId', how='left')

In [103]:
text = "O.G.C. Nice C\u00f4te d'Azur"
text = text.encode().decode('unicode_escape')
print(text)

O.G.C. Nice CÃ´te d'Azur


### Unir datos de jugadores

df_all_info:
- 3603 jugadores
- pero 1164 no tienen equipo asignado (

In [104]:
df_all_info['fullName'] = df_all_info['firstName'] + ' ' + df_all_info['lastName']

In [105]:
df_all_info[df_all_info['firstName'] == "Ferrán"]

,firstName,middleName,lastName,currentTeamId,weight,height,foot,country,cou_code,role,city,name,officialName,wyId,fullName
375,Ferrán,,Torres García,674,77,183,right,Spain,ES,Midfielder,Valencia,Valencia,Valencia Club de Fútbol,674.0,Ferrán Torres García


In [112]:
df_all_info.shape

(3603, 15)

In [107]:
#df_all_info[df_all_info['officialName'].isna() == True]


In [108]:
player_stats.shape

(3164, 30)

In [109]:
df_all_info.merge(player_stats, left_on = 'fullName', right_on = 'player', how='left')

,firstName,middleName,lastName,currentTeamId,weight,height,foot,country,cou_code,role,...,xg,npxg,xg_assist,npxg_xg_assist,xg_per90,xg_assist_per90,xg_xg_assist_per90,npxg_per90,npxg_xg_assist_per90,player
0,Harun,,Tekin,4502,78,187,right,Turkey,TR,Goalkeeper,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Malang,,Sarr,3775,73,182,left,France,FR,Defender,...,0.8,0.8,0.4,1.2,0.05,0.03,0.08,0.05,0.08,Malang Sarr
2,Over,,Mandanda,3772,72,176,,France,FR,Goalkeeper,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Over Mandanda
3,Alfred John Momar,,N'Diaye,683,82,187,right,France,FR,Midfielder,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ibrahima,,Konaté,2975,84,192,right,France,FR,Defender,...,0.3,0.3,0.5,0.8,0.02,0.04,0.06,0.02,0.06,Ibrahima Konaté
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3688,Ali,,Maâloul,16041,72,175,left,Tunisia,TN,Defender,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3689,Carlos Alberto,,Cáceda Oyaguez,15591,76,183,right,Peru,PE,Goalkeeper,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3690,Miguel Gianpierre,,Araujo Blanco,12072,78,179,right,Peru,PE,Defender,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3691,Ahmed Reda,,Tagnaouti,16183,70,182,right,Morocco,MA,Goalkeeper,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Webscraped data graphs

(solo temporales, con mejores métricas usaremos wyscout)

In [110]:
df_events

,Unnamed: 0,eventId,subEventName,playerId,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,...,x_inicio,y_inicio,x_fin,y_fin,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5
0,0,8,Simple pass,25413,2499719,Pass,1609,1H,2.758649,85.0,...,49,49,31.0,78.0,1801.0,NaN,NaN,NaN,NaN,NaN
1,1,8,High pass,370224,2499719,Pass,1609,1H,4.946850,83.0,...,31,78,51.0,75.0,1801.0,NaN,NaN,NaN,NaN,NaN
2,2,8,Head pass,3319,2499719,Pass,1609,1H,6.542188,82.0,...,51,75,35.0,71.0,1801.0,NaN,NaN,NaN,NaN,NaN
3,3,8,Head pass,120339,2499719,Pass,1609,1H,8.143395,82.0,...,35,71,41.0,95.0,1801.0,NaN,NaN,NaN,NaN,NaN
4,4,8,Simple pass,167145,2499719,Pass,1609,1H,10.302366,85.0,...,41,95,72.0,88.0,1801.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628654,628654,8,Simple pass,20623,2565927,Pass,682,2H,2939.077491,85.0,...,66,25,88.0,2.0,1801.0,NaN,NaN,NaN,NaN,NaN
628655,628655,7,Acceleration,122832,2565927,Others on the ball,682,2H,2940.515560,70.0,...,88,2,97.0,21.0,1801.0,NaN,NaN,NaN,NaN,NaN
628656,628656,8,Cross,122832,2565927,Pass,682,2H,2942.098761,80.0,...,97,21,92.0,26.0,401.0,1802.0,NaN,NaN,NaN,NaN
628657,628657,8,Simple pass,40756,2565927,Pass,675,2H,2943.089232,85.0,...,8,74,9.0,56.0,1801.0,NaN,NaN,NaN,NaN,NaN


## VAEP

In [111]:
df_vaep.groupby(by=['first_name', 'last_name', 'team_name', 'player_id', 'type_name']).sum().sort_values(by='vaep_value', ascending=False).head(20)

NameError: name 'df_vaep' is not defined

In [ ]:
df_vaep['type_name'].unique()

In [ ]:
gk_df = df_vaep[df_vaep['type_name'] == 'keeper_save']

In [ ]:
gk_df.groupby(by=['first_name', 'last_name', 'team_name', 'player_id', 'type_name']).sum().sort_values(by='vaep_value', ascending=False).head(20)

In [ ]:
#salvar 

#df_all_info.to_csv('./../../data/wyscout_tabular/players_info_dash.csv')

In [ ]:
#df_events.to_csv('./../../data/wyscout_tabular/all_events.csv')